In [ ]:
# 左边能显示的目录是/content，在里面创建一个名为temp的工作目录
%cd /content
!mkdir temp

In [ ]:
# 1 下载源码
%cd /content/temp
!git clone https://github.com/internlm/InternLM.git

In [ ]:
# 从hf下载InternLM2-Chat-1.8B 模型到自建的/content/model路径（备用）
# %cd /content
# !mkdir model
# !pip install huggingface-hub==0.17.3
# !huggingface-cli download --resume-download internlm/internlm2-chat-1_8b --local-dir /content/model


In [ ]:
# 2 安装依赖
# 由于colab一旦关机，工作目录和系统目录里文件都会删除，所以没必要在工作目录里创建一个虚拟环境
# 因此直接在系统环境里安装InternLM2-Chat-1.8B 模型所需依赖

# !pip install transformers==4.34 # 系统自带transformers-4.41.2能正常运行该模型，强行更换版本要restart session一次，不优雅
# !pip install psutil==5.9.8 #系统环境已经有5.9.5版本的，且5.9.5版本能正常运行该项目，升级要restart session
!pip install accelerate==0.24.1
!pip install streamlit==1.32.2
# !pip install matplotlib==3.8.3 # 系统环境已经有matplotlib==2.1.5，升级要restart session
!pip install modelscope==1.9.5
!pip install sentencepiece==0.1.99

In [ ]:
# 3 下载对话模型，运行后在命令行里进行对话
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM


model_name_or_path = "internlm/internlm2-chat-1_8b"
# 直接填写hf的相应模型仓库名，自动下载模型到InternLM源代码的模型存放目录

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True, device_map='cuda:0')
model = AutoModelForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True, torch_dtype=torch.bfloat16, device_map='cuda:0')
model = model.eval()

system_prompt = """You are an AI assistant whose name is InternLM (书生·浦语).
- InternLM (书生·浦语) is a conversational language model that is developed by Shanghai AI Laboratory (上海人工智能实验室). It is designed to be helpful, honest, and harmless.
- InternLM (书生·浦语) can understand and communicate fluently in the language chosen by the user such as English and 中文.
"""

messages = [(system_prompt, '')]

print("=============Welcome to InternLM chatbot, type 'exit' to exit.=============")

while True:
    input_text = input("\nUser  >>> ")
    input_text = input_text.replace(' ', '')
    if input_text == "exit":
        break

    length = 0
    for response, _ in model.stream_chat(tokenizer, input_text, messages):
        if response is not None:
            print(response[length:], flush=True, end="")
            length = len(response)


In [ ]:
# 4.1使用ngrok进行内网穿透，用它帮助我们用自己笔记本来访问colab本地的127.0.0.1:6006
!pip install pyngrok
from pyngrok import ngrok

# 此处字符串里的token就是ngrok的个人密钥
# 去https://ngrok.com/注册后，左栏有个Your Authtoken，复制了粘贴在下方即可
YOUR_NGROK_AUTHTOKEN = "2ZDjz2GRqYOh66SkdzOjF8BmtIJ_2QyLsvabaLwvni1c7qKqU"

ngrok.set_auth_token(YOUR_NGROK_AUTHTOKEN)

# 完成colab端口的穿透，可以在公网访问colab本地的6006端口
public_url = ngrok.connect(6006)
print("Tunnel URL:", public_url)


In [ ]:
# 4.2 web demo里进行模型对话
%cd /content/temp/InternLM/chat
# web_demo.py里默认用7b的模型，需要替换为1.8b的模型，title也替换为1.8b
!sed -i 's/internlm2-chat-7b/internlm2-chat-1_8b/g' /content/temp/InternLM/chat/web_demo.py
!sed -i 's/InternLM2-Chat-7B/InternLM2-Chat-1.8B/g' /content/temp/InternLM/chat/web_demo.py

#相对路径替换为绝对路径，解决No such file or directory: 'assets/user.png'的问题
!sed -i "s#user_avator = 'assets/user.png'#user_avator = '/content/temp/InternLM/assets/user.png'#g" /content/temp/InternLM/chat/web_demo.py
!sed -i "s#robot_avator = 'assets/robot.png'#robot_avator = '/content/temp/InternLM/assets/robot.png'#g" /content/temp/InternLM/chat/web_demo.py

!streamlit run web_demo.py --server.address 127.0.0.1 --server.port 6006
# 等下方出现URL: http://127.0.0.1:6006之后，点击上方的Tunnel URL: NgrokTunnel:"https://xxxxx.app"里的网址，
# 然后点击Visit site进入web demo的网页，之后等待模型加载完成，即可开始对话